In [1]:
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

In [10]:
raw_df = pd.read_csv('crypto_data.csv')
raw_df.sample(10)

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
811,XIOS,Xios,Scrypt,True,PoW/PoS,2.149688e+06,21000000
489,EGO,EGOcoin,PoS,True,PoS,NaN,60000000
458,ZNE,ZoneCoin,Scrypt,True,PoW/PoS,2.581970e+06,21000000
40,BET,BetaCoin,SHA-256,True,PoW,0.000000e+00,32000000
201,XCSH,Xcash,Scrypt,False,PoW/PoS,NaN,2400000
328,BLITZ,BlitzCoin,X13,True,PoS,0.000000e+00,3852156
290,XMS,Megastake,X11,True,PoW/PoS,NaN,0
61,CAT,Catcoin,Scrypt,True,PoW,7.219957e+06,21000000
439,EXB,ExaByte (EXB),SHA-256,True,PoW,NaN,500000000
566,NPC,NPCcoin,SHA-256D,True,PoW/PoS,0.000000e+00,7000000000


In [8]:
traded_df = raw_df.loc[raw_df['IsTrading'] == True]
traded_df.sample(10)

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
971,ZEL,Zelcash,Equihash,True,PoW/PoS,0.000000e+00,210000000
986,CHARM,Charm Coin,NeoScrypt,True,PoW,NaN,250000000
384,FLX,Flash,Scrypt,True,PoW/PoS,NaN,1000000
110,HYP,HyperStake,X11,True,PoS,1.678520e+09,0
620,LENIN,LeninCoin,SHA-256D,True,PoW,NaN,100000000
551,DRM8,Dream8Coin,Scrypt,True,PoW,NaN,88800000000
336,HODL,HOdlcoin,1GB AES Pattern Search,True,PoW,1.144895e+07,81962100
851,DAXX,DaxxCoin,Ethash,True,PoW,5.208918e+08,10000000000
727,PZM,Prizm,SHA-256,True,PoS,NaN,600000000000000
80,DVC,DevCoin,SHA-256,True,PoW,1.877386e+10,21000000000
